<a href="https://colab.research.google.com/github/huangchengwu/NLP_bert/blob/master/faiss_text_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━ 385.1/731.7 MB 73.4 MB/s eta 0:00:05

In [ ]:
!pip install faiss-cpu sentence-transformers





In [ ]:
!pip install --force-reinstall numpy==1.26.4





  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import torch
print(torch.cuda.is_available())  # 如果返回 True，说明有可用的 GPU


True


In [ ]:
# Use a pipeline as a high-level helper
#未训练前结果
from transformers import pipeline

messages = [
    {"role": "user", "content": "什么是向量数据库？"},
]
pipe = pipeline("text-generation", model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
pipe(messages)

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': '什么是向量数据库？'},
   {'role': 'assistant', 'content': '嗯，向量数据库是什么？我之前听说过数据库，但向量数据库听起来有点复杂。'}]}]

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# 定义并加载嵌入模型
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# 动态获取实际嵌入维度
sample_embedding = model.encode(["测试"])[0]
dim = len(sample_embedding)

# 初始化 FAISS 向量数据库
index = faiss.IndexFlatL2(dim)

# 示例文档列表
documents = [
    "FAISS 是一个高效的向量数据库，专门用于近似最近邻搜索。",
    "SentenceTransformers 是一个用于生成文本嵌入的库，适合用在语义搜索中。",
    "向量数据库可以用于存储大规模的文本数据，并且可以高效地进行相似性搜索。",
    "MySQL 是一个关系型数据库管理系统，可以存储和查询结构化数据。",
    "机器学习可以用于文本分类、回归、聚类等任务。"
    "黄承武是东平人，东平镇，年龄是30呢",
]

# 将文档嵌入并添加到 FAISS 向量数据库
def embed_and_upsert_document(index, document_id, document_content):
    embedding = model.encode([document_content])[0]
    embedding = np.array(embedding, dtype=np.float32).reshape(1, -1)
    index.add(embedding)
    print(f"Document {document_id} added to FAISS.")

# 添加文档
for idx, doc in enumerate(documents):
    embed_and_upsert_document(index, idx, doc)

print("All documents added to FAISS.")

# 查询 FAISS
def query_faiss(index, query_text, n_results=3):
    query_embedding = model.encode([query_text])[0]
    query_embedding = np.array(query_embedding, dtype=np.float32).reshape(1, -1)
    distances, indices = index.search(query_embedding, n_results)
    return distances, indices
query_text = "谁是黄承武"


# 示例查询                                                                                                                                                                                                                                                                                            = "什么是向量数据库？"
distances, indices = query_faiss(index, query_text)

print("==== FAISS 查询结果 ====")
for i in range(len(indices[0])):
    doc_idx = indices[0][i]
    print(f"最相关文档 {i+1}: {documents[doc_idx]} (距离: {distances[0][i]:.4f})")

# model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model_name="Qwen/Qwen1.5-7B-Chat"



pipe = pipeline("text-generation", model=model_name)
pipe(messages)


# 拼接检索结果作为上下文
retrieved_context = "\n".join([documents[i] for i in indices[0]])



prompt = f"""你是一个问答助手，请仅根据给出的背景信息回答问题，不要编造信息，不要引用其他来源，也不要发挥。

背景信息：
{retrieved_context}

问题：{query_text}
请用一句话回答："""




print("prompt",prompt)
# 生成回答
response = pipe(prompt,
                max_new_tokens=200,
                do_sample=True,          # 开启采样
                temperature=0.7,         # 控制“多样性”
                top_k=50,                # 限制候选范围
                repetition_penalty=1.2   # 惩罚重复
               )


print(response[0]['generated_text'])



Document 0 added to FAISS.
Document 1 added to FAISS.
Document 2 added to FAISS.
Document 3 added to FAISS.
Document 4 added to FAISS.
All documents added to FAISS.
==== FAISS 查询结果 ====
最相关文档 1: 向量数据库可以用于存储大规模的文本数据，并且可以高效地进行相似性搜索。 (距离: 33.8032)
最相关文档 2: 机器学习可以用于文本分类、回归、聚类等任务。黄承武是东平人，东平镇，年龄是30呢 (距离: 38.6464)
最相关文档 3: MySQL 是一个关系型数据库管理系统，可以存储和查询结构化数据。 (距离: 54.3019)


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/31.7k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00001-of-00004.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.54G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]